In [1]:
import os
import sys


os.chdir("/teamspace/studios/this_studio/nurse-helper-chatbot")


sys.path.append(os.getcwd())

In [2]:
from llm import llm
from graph import graph
from langchain.chains import GraphCypherQAChain
from langchain.prompts.prompt import PromptTemplate


CYPHER_GENERATION_TEMPLATE = """Task: Generate a Cypher statement to query a graph database.
Instructions:
- Use only the provided relationship types and properties in the schema.
- Do not use any other relationship types or properties that are not provided.
- Ensure that the generated Cypher statements are syntactically correct and relevant to the question.
- Do not include any explanations or apologies in your responses.
- Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
- Do not include any text except the generated Cypher statement.

Do not return entire nodes or embedding properties.

Fine Tuning:

Example Cypher Statements:

1. To find checkup details for a patient Using Patient ID, Name, or Identity Number
```
MATCH (c:checkup)<-[:has]-(p:patient)
WHERE p.Patient_ID = $patientId 
   OR p.Name = $patientName
   OR p.Identity_number = $identityNumber
RETURN c.Checkup_ID AS CheckupID, 
       c.Checkup_Status AS CheckupStatus, 
       c.Admission_Date AS AdmissionDate, 
       c.Discharge_Date AS DischargeDate,
       c.`Length_of_Stay (LOS)` AS LengthofStay,
       c.Discharge_Condition AS DischargeCondition, 
       c.Diagnosis AS Diagnosis, 
       c.Medication AS Medication, 
       c.Room_Number AS RoomNumber;
```

2. To find all checkups attended by a specific physician and which patient they belong to
```
MATCH (p:patient)-[:has]->(c:checkup)<-[:attend]-(ph:physician)
WHERE ph.Physician_ID = $physicianId OR ph.Physician_Name = $physicianName
RETURN c.Checkup_ID AS CheckupID, 
       c.Admission_Type AS AdmissionType, 
       c.Checkup_Status AS CheckupStatus, 
       c.Diagnosis AS Diagnosis, 
       c.Treatment AS Treatment, 
       c.Specialisation AS Specialisation, 
       c.Patient_ID AS PatientID, 
       c.Room_Number AS RoomNumber,
       p.Name AS PatientName, 
       ph.Specialization AS PhysicianSpecialisation;
```

3. To find checkups conducted in a specific room with checkup and patient details
```
MATCH (p:patient)-[:has]->(c:checkup)
WHERE c.Room_Number = "A314"
RETURN c.Checkup_ID AS CheckupID, 
       c.Admission_Type AS AdmissionType, 
       c.Checkup_Status AS CheckupStatus, 
       c.Diagnosis AS Diagnosis, 
       c.Medication AS Medication,  
       p.Identity_number AS IdentityNumber, 
       p.Name AS PatientName;
```

4. To find checkups for a specific physician based on valid test results or admission types with patient details
```
MATCH (p:patient)-[:has]->(c:checkup)<-[:attend]-(ph:physician)
WHERE (c.Admission_Type IN ["Emergency", "Urgent", "Elective"] AND c.Admission_Type = $admissionType)
   OR (c.Test_Results IN ["Abnormal", "Normal", "Inconclusive"] AND c.Test_Results = $testResults)
   AND ph.Physician_Name = $physicianName
RETURN c.Checkup_ID AS CheckupID, 
       c.Diagnosis AS Diagnosis, 
       c.Treatment AS Treatment, 
       c.Test_Results AS TestResults, 
       c.Admission_Type AS AdmissionType, 
       c.Medication AS Medication, 
       p.Name AS PatientName,  
       ph.Physician_Name AS PhysicianName, 
       ph.Specialization AS Specialisation;


Schema:
{schema}

Question:
{question}

Cypher Query:
"""


checkup_cypher_prompt = PromptTemplate.from_template(CYPHER_GENERATION_TEMPLATE)

checkup_cypher_qa = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    return_intermediate_steps=True,
    verbose=True,
    allow_dangerous_requests=True,
    validate_cypher=True,
    function_response_system="Response based on the Cypher Query and context",
    cypher_prompt=checkup_cypher_prompt
)

/teamspace/studios/this_studio/nurse-helper-chatbot/graph.py:7: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


In [8]:
# Example query to invoke the chain
## What are the details of checkups conducted in room A314?, Show me the checkups performed in room A314 along with patient details.
## What checkups attended by Dr. Aisyah binti Kamaruddin include patient details?, Show all checkups attended by the physician with ID 261.
## What checkups has the patient with ID 321 undergone?, Show the checkups for the patient named Cheryl Lee.
## Retrieve checkups for Dr. Ng Yee Siang that involve emergencies or abnormal test results., (not work sometime) What emergency checkups has Dr. Ng Yee Siang attended?, List abnormal test result checkups handled by Dr. Ng Yee Siang., 

query = "What checkups attended by Dr. Aisyah binti Kamaruddin include patient details?"

# Invoke the chain with the complete input data
response = checkup_cypher_qa.invoke(query)

response



> Entering new GraphCypherQAChain chain...


Generated Cypher:

MATCH (p:patient)-[:has]->(c:checkup)<-[:attend]-(ph:physician)
WHERE ph.Physician_Name = "Dr. Aisyah binti Kamaruddin"
RETURN c.Checkup_ID AS CheckupID, 
       c.Admission_Type AS AdmissionType, 
       c.Checkup_Status AS CheckupStatus, 
       c.Diagnosis AS Diagnosis, 
       c.Treatment AS Treatment, 
       c.Specialisation AS Specialisation, 
       p.Patient_ID AS PatientID, 
       p.Name AS PatientName;

Full Context:
[{'CheckupID': 619, 'AdmissionType': 'Urgent', 'CheckupStatus': 'Open', 'Diagnosis': 'Cancer', 'Treatment': 'Antibiotic course', 'Specialisation': 'Oncology', 'PatientID': 386, 'PatientName': 'Lee Hwa Yan'}]

> Finished chain.


{'query': 'What checkups attended by Dr. Aisyah binti Kamaruddin include patient details?',
 'result': "I don't know the answer.",
 'intermediate_steps': [{'query': '\nMATCH (p:patient)-[:has]->(c:checkup)<-[:attend]-(ph:physician)\nWHERE ph.Physician_Name = "Dr. Aisyah binti Kamaruddin"\nRETURN c.Checkup_ID AS CheckupID, \n       c.Admission_Type AS AdmissionType, \n       c.Checkup_Status AS CheckupStatus, \n       c.Diagnosis AS Diagnosis, \n       c.Treatment AS Treatment, \n       c.Specialisation AS Specialisation, \n       p.Patient_ID AS PatientID, \n       p.Name AS PatientName;\n'},
  {'context': [{'CheckupID': 619,
     'AdmissionType': 'Urgent',
     'CheckupStatus': 'Open',
     'Diagnosis': 'Cancer',
     'Treatment': 'Antibiotic course',
     'Specialisation': 'Oncology',
     'PatientID': 386,
     'PatientName': 'Lee Hwa Yan'}]}]}

In [6]:
response = hospital_cypher_qa.invoke({"query": "Retrieve checkups for Dr. Ng Yee Siang that involve emergencies or abnormal test results."})
response



> Entering new GraphCypherQAChain chain...


Generated Cypher:

MATCH (p:patient)-[:has]->(c:checkup)<-[:attend]-(ph:physician)
WHERE (c.Admission_Type = "Emergency" OR c.Test_Results = "Abnormal")
   AND ph.Physician_Name = "Dr. Ng Yee Siang"
RETURN c.Checkup_ID AS CheckupID, 
       c.Admission_Type AS AdmissionType, 
       c.Checkup_Status AS CheckupStatus, 
       c.Diagnosis AS Diagnosis, 
       c.Treatment AS Treatment, 
       c.Test_Results AS TestResults, 
       c.Medication AS Medication, 
       p.Name AS PatientName,  
       ph.Physician_Name AS PhysicianName, 
       ph.Specialization AS Specialisation;

Full Context:
[{'CheckupID': 190, 'AdmissionType': 'Emergency', 'CheckupStatus': 'Open', 'Diagnosis': 'Arthritis', 'Treatment': 'IV fluids', 'TestResults': 'Inconclusive', 'Medication': 'Prednisone', 'PatientName': 'Faris Alif', 'PhysicianName': 'Dr. Ng Yee Siang', 'Specialisation': 'Orthopedics'}, {'CheckupID': 246, 'AdmissionType': 'Emergency', 'CheckupStatus': 'Open', 'Diagnosis': 'Back pain', 'Treatment': 'Bl

{'query': 'Retrieve checkups for Dr. Ng Yee Siang that involve emergencies or abnormal test results.',
 'result': 'CheckupID: 190, 246 for Dr. Ng Yee Siang involve emergencies or abnormal test results.',
 'intermediate_steps': [{'query': '\nMATCH (p:patient)-[:has]->(c:checkup)<-[:attend]-(ph:physician)\nWHERE (c.Admission_Type = "Emergency" OR c.Test_Results = "Abnormal")\n   AND ph.Physician_Name = "Dr. Ng Yee Siang"\nRETURN c.Checkup_ID AS CheckupID, \n       c.Admission_Type AS AdmissionType, \n       c.Checkup_Status AS CheckupStatus, \n       c.Diagnosis AS Diagnosis, \n       c.Treatment AS Treatment, \n       c.Test_Results AS TestResults, \n       c.Medication AS Medication, \n       p.Name AS PatientName,  \n       ph.Physician_Name AS PhysicianName, \n       ph.Specialization AS Specialisation;\n'},
  {'context': [{'CheckupID': 190,
     'AdmissionType': 'Emergency',
     'CheckupStatus': 'Open',
     'Diagnosis': 'Arthritis',
     'Treatment': 'IV fluids',
     'TestResults

In [10]:
from langchain.chains import SequentialChain
from langchain_core.output_parsers import StrOutputParser
# Initialize the output parser
output_parser = StrOutputParser()

query = "Retrieve checkups for Dr. Ng Yee Siang that involve emergencies or abnormal test results."
cypher_response = hospital_cypher_qa.invoke({"query": query})

# Parse the output
parsed_response = output_parser.parse(cypher_response)

# Print the final response
print(parsed_response)



> Entering new GraphCypherQAChain chain...


Generated Cypher:

MATCH (p:patient)-[:has]->(c:checkup)<-[:attend]-(ph:physician)
WHERE (c.Admission_Type = "Emergency" OR c.Test_Results = "Abnormal")
AND ph.Physician_Name = "Dr. Ng Yee Siang"
RETURN c.Checkup_ID AS CheckupID, 
       c.Diagnosis AS Diagnosis, 
       c.Treatment AS Treatment, 
       c.Test_Results AS TestResults, 
       c.Admission_Type AS AdmissionType, 
       c.Medication AS Medication, 
       p.Name AS PatientName,  
       ph.Physician_Name AS PhysicianName, 
       ph.Specialization AS Specialisation;

Full Context:
[{'CheckupID': 190, 'Diagnosis': 'Arthritis', 'Treatment': 'IV fluids', 'TestResults': 'Inconclusive', 'AdmissionType': 'Emergency', 'Medication': 'Prednisone', 'PatientName': 'Faris Alif', 'PhysicianName': 'Dr. Ng Yee Siang', 'Specialisation': 'Orthopedics'}, {'CheckupID': 246, 'Diagnosis': 'Back pain', 'Treatment': 'Blood pressure management', 'TestResults': 'Abnormal', 'AdmissionType': 'Emergency', 'Medication': 'Ceftriaxone', 'PatientName':

In [11]:
def format_cypher_output(response):
    # Extracting the result and intermediate steps
    result = response.get('result', 'No result found.')
    intermediate_steps = response.get('intermediate_steps', [])

    # Start building the output string
    output_string = f"Query: {response['query']}\n"
    output_string += f"Result: {result}\n\n"
    output_string += "Details of Checkups:\n"

    # Loop through the intermediate steps to extract context
    for step in intermediate_steps:
        context = step.get('context', [])
        for checkup in context:
            output_string += (
                f"Checkup ID: {checkup['CheckupID']}, "
                f"Patient: {checkup['PatientName']}, "
                f"Diagnosis: {checkup['Diagnosis']}, "
                f"Treatment: {checkup['Treatment']}, "
                f"Test Results: {checkup['TestResults']}, "
                f"Admission Type: {checkup['AdmissionType']}, "
                f"Medication: {checkup['Medication']}, "
                f"Physician: {checkup['PhysicianName']}, "
                f"Specialization: {checkup['Specialisation']}\n"
            )

    return output_string


In [12]:
# Assuming 'response' is the output you provided
formatted_output = format_cypher_output(response)
print(formatted_output)


Query: Retrieve checkups for Dr. Ng Yee Siang that involve emergencies or abnormal test results.
Result: CheckupID: 190, 246 for Dr. Ng Yee Siang involve emergencies or abnormal test results.

Details of Checkups:
Checkup ID: 190, Patient: Faris Alif, Diagnosis: Arthritis, Treatment: IV fluids, Test Results: Inconclusive, Admission Type: Emergency, Medication: Prednisone, Physician: Dr. Ng Yee Siang, Specialization: Orthopedics
Checkup ID: 246, Patient: Azizul Rahim, Diagnosis: Back pain, Treatment: Blood pressure management, Test Results: Abnormal, Admission Type: Emergency, Medication: Ceftriaxone, Physician: Dr. Ng Yee Siang, Specialization: Orthopedics
Checkup ID: 399, Patient: Mohd Isa, Diagnosis: Arthritis, Treatment: Surgery, Test Results: Normal, Admission Type: Emergency, Medication: Methotrexate, Physician: Dr. Ng Yee Siang, Specialization: Orthopedics

